# FAISS向量存储

## 1. 基础创建和初始化

In [19]:
# 基础FAISS向量存储创建
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_core.documents import Document

# 初始化嵌入模型
embeddings = OllamaEmbeddings(
    base_url="http://localhost:11434",
    model="nomic-embed-text:latest"
)

# 准备文档数据
documents = [
    Document(
        page_content="人工智能是计算机科学的一个分支，致力于创建智能机器",
        metadata={"source": "ai_intro.txt", "category": "technology"}
    ),
    Document(
        page_content="机器学习是人工智能的子集，使计算机能够从数据中学习",
        metadata={"source": "ml_basics.txt", "category": "technology"}
    ),
    Document(
        page_content="深度学习使用神经网络来模拟人脑的工作方式",
        metadata={"source": "dl_guide.txt", "category": "technology"}
    )
]

# 方法1：从文档创建
faiss_vectorstore = FAISS.from_documents(documents, embeddings)

# # 方法2：从文本创建
# texts = ["人工智能技术", "机器学习算法", "深度学习网络"]
# metadatas = [{"id": 1}, {"id": 2}, {"id": 3}]
# faiss_vectorstore = FAISS.from_texts(texts, embeddings, metadatas)

## 2. 保存和加载索引

In [20]:
# 保存FAISS索引到本地
index_path = "faiss_index"
faiss_vectorstore.save_local(index_path)
print(f"✅ FAISS索引已保存到 {index_path}")

# 加载已保存的索引
loaded_vectorstore = FAISS.load_local(
    index_path,
    embeddings,
    allow_dangerous_deserialization=True  # 允许反序列化
)
print("✅ FAISS索引加载成功")

# 验证加载结果
# test_results = loaded_vectorstore.similarity_search("人工智能", k=1)
test_results = loaded_vectorstore.similarity_search("") # 空则查询所有文档
print(test_results)
print(f"加载验证：找到 {len(test_results)} 个文档")

✅ FAISS索引已保存到 faiss_index
✅ FAISS索引加载成功
[Document(id='b85e8ec4-f153-4d49-aca9-51fa3fcf0978', metadata={'source': 'ai_intro.txt', 'category': 'technology'}, page_content='人工智能是计算机科学的一个分支，致力于创建智能机器'), Document(id='cb7eebdd-e276-4e09-bf92-6e2b97e807eb', metadata={'source': 'dl_guide.txt', 'category': 'technology'}, page_content='深度学习使用神经网络来模拟人脑的工作方式'), Document(id='707f40c2-31a4-45fe-9b56-ab87ae9aecfc', metadata={'source': 'ml_basics.txt', 'category': 'technology'}, page_content='机器学习是人工智能的子集，使计算机能够从数据中学习')]
加载验证：找到 3 个文档


## 3. 相似性搜索操作

In [22]:
# 3.1 基础相似性搜索
query = "什么是人工智能技术？"
similar_docs = faiss_vectorstore.similarity_search(query, k=3)

print(f"查询: '{query}'")
for i, doc in enumerate(similar_docs):
    print(f"{i+1}. {doc.page_content}")
    print(f"   元数据: {doc.metadata}")

# 3.2 带分数的相似性搜索
similar_docs_with_scores = faiss_vectorstore.similarity_search_with_score(query, k=3)

print("带分数的搜索结果:")
for i, (doc, score) in enumerate(similar_docs_with_scores):
    print(f"{i+1}. 分数: {score:.4f}")
    print(f"   内容: {doc.page_content}")
    print(f"   来源: {doc.metadata.get('source', 'unknown')}")

# # 3.3 基于阈值的搜索
# threshold = 0.8
# similar_docs_threshold = faiss_vectorstore.similarity_search_with_score_threshold(
#     query,
#     score_threshold=threshold
# )
#
# print(f"阈值 {threshold} 以上的文档:")
# for doc, score in similar_docs_threshold:
#     print(f"分数: {score:.4f} - {doc.page_content[:60]}...")

查询: '什么是人工智能技术？'
1. 人工智能是计算机科学的一个分支，致力于创建智能机器
   元数据: {'source': 'ai_intro.txt', 'category': 'technology'}
2. 机器学习是人工智能的子集，使计算机能够从数据中学习
   元数据: {'source': 'ml_basics.txt', 'category': 'technology'}
3. 深度学习使用神经网络来模拟人脑的工作方式
   元数据: {'source': 'dl_guide.txt', 'category': 'technology'}
带分数的搜索结果:
1. 分数: 0.3241
   内容: 人工智能是计算机科学的一个分支，致力于创建智能机器
   来源: ai_intro.txt
2. 分数: 0.3984
   内容: 机器学习是人工智能的子集，使计算机能够从数据中学习
   来源: ml_basics.txt
3. 分数: 0.6661
   内容: 深度学习使用神经网络来模拟人脑的工作方式
   来源: dl_guide.txt


## 4. 文档管理操作

In [18]:
# 4.1 添加新文档
new_documents = [
    Document(
        page_content="量子计算是一种利用量子力学原理的计算方式",
        metadata={"source": "quantum.txt", "category": "technology"}
    ),
    Document(
        page_content="区块链技术提供了去中心化的数据存储方案",
        metadata={"source": "blockchain.txt", "category": "technology"}
    )
]

# 添加文档并获取ID
doc_ids = faiss_vectorstore.add_documents(new_documents)
print(f"✅ 已添加 {len(new_documents)} 个新文档，ID: {doc_ids}")

# 4.2 添加文本
new_texts = ["自然语言处理技术", "计算机视觉应用"]
new_metadatas = [{"type": "nlp"}, {"type": "cv"}]
text_ids = faiss_vectorstore.add_texts(new_texts, new_metadatas)
print(f"✅ 已添加 {len(new_texts)} 个新文本，ID: {text_ids}")

# 4.3 删除文档（如果支持）
try:
    faiss_vectorstore.delete([doc_ids[0]])
    print("✅ 文档删除成功")
except Exception as e:
    print(f"删除操作不支持: {e}")

✅ 已添加 2 个新文档，ID: ['27806a87-e8f3-4c60-8be3-1a68759e517d', 'd05964ce-d18e-497e-86cc-0aadb3f625df']
✅ 已添加 2 个新文本，ID: ['9d839ddf-6af9-4fe2-99ad-f35168b45eee', '8d1a8c2f-dd53-43bb-b2cd-bc515a2e6e78']
✅ 文档删除成功


## 5. 高级索引类型

In [23]:
# 5.1 IVF索引（适合大规模数据）
try:
    faiss_ivf = FAISS.from_documents(
        documents,
        embeddings,
        index_type="IVF",
        nlist=10  # 聚类中心数量
    )

    ivf_results = faiss_ivf.similarity_search("机器学习算法", k=2)
    print("IVF索引搜索结果:")
    for doc in ivf_results:
        print(f"- {doc.page_content}")

except Exception as e:
    print(f"IVF索引创建失败: {e}")

# 5.2 HNSW索引（高精度搜索）
try:
    faiss_hnsw = FAISS.from_documents(
        documents,
        embeddings,
        index_type="HNSW",
        M=16,  # 连接数
        efConstruction=200  # 构建时的搜索参数
    )

    hnsw_results = faiss_hnsw.similarity_search("深度学习", k=2)
    print("HNSW索引搜索结果:")
    for doc in hnsw_results:
        print(f"- {doc.page_content}")

except Exception as e:
    print(f"HNSW索引创建失败: {e}")

IVF索引创建失败: FAISS.__init__() got an unexpected keyword argument 'index_type'
HNSW索引创建失败: FAISS.__init__() got an unexpected keyword argument 'index_type'


## 6. 向量存储合并

In [24]:
# 6.1 创建两个独立的向量存储
docs1 = [
    Document(page_content="第一个存储的文档1", metadata={"store": "store1"}),
    Document(page_content="第一个存储的文档2", metadata={"store": "store1"})
]

docs2 = [
    Document(page_content="第二个存储的文档1", metadata={"store": "store2"}),
    Document(page_content="第二个存储的文档2", metadata={"store": "store2"})
]

store1 = FAISS.from_documents(docs1, embeddings)
store2 = FAISS.from_documents(docs2, embeddings)

# 6.2 合并向量存储
store1.merge_from(store2)
print("✅ 向量存储合并完成")

# 6.3 验证合并结果
all_results = store1.similarity_search("文档", k=4)
print(f"合并后总文档数: {len(all_results)}")
for doc in all_results:
    print(f"- {doc.page_content} (来源: {doc.metadata['store']})")

✅ 向量存储合并完成
合并后总文档数: 4
- 第一个存储的文档2 (来源: store1)
- 第一个存储的文档1 (来源: store1)
- 第二个存储的文档1 (来源: store2)
- 第二个存储的文档2 (来源: store2)


## 7. 批量操作和性能优化

In [25]:
# 7.1 批量添加文档
batch_docs = [
    Document(
        page_content=f"批量文档{i}：这是第{i}个批量添加的文档",
        metadata={"batch": True, "id": i, "category": f"batch_{i%3}"}
    )
    for i in range(50)  # 创建50个文档
]

# 批量添加
start_time = time.time()
batch_ids = faiss_vectorstore.add_documents(batch_docs)
end_time = time.time()

print(f"✅ 批量添加 {len(batch_docs)} 个文档")
print(f"耗时: {end_time - start_time:.4f} 秒")
print(f"平均每个文档: {(end_time - start_time) / len(batch_docs):.6f} 秒")

# 7.2 批量搜索测试
queries = ["批量文档", "人工智能", "机器学习", "深度学习"]

start_time = time.time()
for query in queries:
    results = faiss_vectorstore.similarity_search(query, k=3)
    print(f"查询 '{query}': {len(results)} 个结果")

end_time = time.time()
print(f"批量搜索耗时: {end_time - start_time:.4f} 秒")

✅ 批量添加 50 个文档
耗时: 1.2681 秒
平均每个文档: 0.025361 秒
查询 '批量文档': 3 个结果
查询 '人工智能': 3 个结果
查询 '机器学习': 3 个结果
查询 '深度学习': 3 个结果
批量搜索耗时: 0.1122 秒


## 8. 获取所有文档

In [26]:
def get_all_documents_from_faiss(vectorstore, max_docs=1000):
    """获取FAISS中的所有文档"""
    try:
        # 方法1：使用空查询
        all_docs = vectorstore.similarity_search("", k=max_docs)
        return all_docs
    except:
        try:
            # 方法2：使用通用词汇
            all_docs = vectorstore.similarity_search("文档", k=max_docs)
            return all_docs
        except:
            # 方法3：使用docstore（如果可用）
            if hasattr(vectorstore, 'docstore'):
                return list(vectorstore.docstore._dict.values())
            return []

# 获取所有文档
all_documents = get_all_documents_from_faiss(faiss_vectorstore)
print(f"总文档数量: {len(all_documents)}")

# 按类别统计
category_count = {}
for doc in all_documents:
    category = doc.metadata.get('category', 'unknown')
    category_count[category] = category_count.get(category, 0) + 1

print("文档类别统计:")
for category, count in category_count.items():
    print(f"- {category}: {count} 个文档")

总文档数量: 53
文档类别统计:
- batch_0: 17 个文档
- batch_1: 17 个文档
- batch_2: 16 个文档
- technology: 3 个文档


## 9. 转换为检索器

In [27]:
# 9.1 基础检索器
retriever = faiss_vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

# 使用检索器
query = "人工智能的应用"
retrieved_docs = retriever.invoke(query)

print(f"检索器结果 (查询: '{query}'):")
for i, doc in enumerate(retrieved_docs):
    print(f"{i+1}. {doc.page_content[:80]}...")

# 9.2 带分数阈值的检索器
threshold_retriever = faiss_vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.5, "k": 5}
)

threshold_results = threshold_retriever.invoke("机器学习")
print(f"阈值检索器结果: {len(threshold_results)} 个文档")

# 9.3 MMR检索器（最大边际相关性）
mmr_retriever = faiss_vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, "fetch_k": 10, "lambda_mult": 0.5}
)

mmr_results = mmr_retriever.invoke("深度学习技术")
print(f"MMR检索器结果: {len(mmr_results)} 个文档")

检索器结果 (查询: '人工智能的应用'):
1. 人工智能是计算机科学的一个分支，致力于创建智能机器...
2. 机器学习是人工智能的子集，使计算机能够从数据中学习...
3. 深度学习使用神经网络来模拟人脑的工作方式...
阈值检索器结果: 3 个文档
MMR检索器结果: 3 个文档


## 10. 性能监控和调试

In [28]:
import time

def benchmark_faiss_operations(vectorstore, test_queries, num_runs=5):
    """FAISS操作性能基准测试"""
    print("=" * 50)
    print("FAISS性能基准测试")
    print("=" * 50)

    # 搜索性能测试
    search_times = []
    for _ in range(num_runs):
        start_time = time.time()
        for query in test_queries:
            results = vectorstore.similarity_search(query, k=5)
        end_time = time.time()
        search_times.append(end_time - start_time)

    avg_search_time = sum(search_times) / len(search_times)
    print(f"平均搜索时间: {avg_search_time:.4f} 秒")
    print(f"每个查询平均时间: {avg_search_time / len(test_queries):.6f} 秒")

    # 添加文档性能测试
    test_docs = [
        Document(page_content=f"性能测试文档{i}", metadata={"test": True})
        for i in range(10)
    ]

    add_times = []
    for _ in range(num_runs):
        start_time = time.time()
        vectorstore.add_documents(test_docs)
        end_time = time.time()
        add_times.append(end_time - start_time)

    avg_add_time = sum(add_times) / len(add_times)
    print(f"平均添加时间: {avg_add_time:.4f} 秒")
    print(f"每个文档平均添加时间: {avg_add_time / len(test_docs):.6f} 秒")

# 运行性能测试
test_queries = ["人工智能", "机器学习", "深度学习", "自然语言处理"]
benchmark_faiss_operations(faiss_vectorstore, test_queries)

FAISS性能基准测试
平均搜索时间: 0.1369 秒
每个查询平均时间: 0.034230 秒
平均添加时间: 0.1604 秒
每个文档平均添加时间: 0.016039 秒


In [48]:

def faiss_vectorstore_example():
    """FAISS向量存储详细示例"""
    print("=" * 60)
    print("1. FAISS向量存储示例")
    print("=" * 60)

    try:
        # 初始化嵌入模型
        # embeddings = HuggingFaceEmbeddings(
        #     model_name="sentence-transformers/all-MiniLM-L6-v2"
        # )
        embeddings = OllamaEmbeddings(
        base_url="http://localhost:11434",
        model="nomic-embed-text:latest",  # 推荐的嵌入模型
        )

        # 准备文档
        documents = [
            Document(
                page_content="人工智能是计算机科学的一个分支，致力于创建智能机器",
                metadata={"source": "ai_intro.txt", "category": "technology", "date": "2024-01-01"}
            ),
            Document(
                page_content="机器学习是人工智能的子集，使计算机能够从数据中学习",
                metadata={"source": "ml_basics.txt", "category": "technology", "date": "2024-01-02"}
            ),
            Document(
                page_content="深度学习使用神经网络来模拟人脑的工作方式",
                metadata={"source": "dl_guide.txt", "category": "technology", "date": "2024-01-03"}
            ),
            Document(
                page_content="自然语言处理让计算机理解和生成人类语言",
                metadata={"source": "nlp_overview.txt", "category": "technology", "date": "2024-01-04"}
            ),
            Document(
                page_content="计算机视觉技术可以识别和分析图像中的内容",
                metadata={"source": "cv_intro.txt", "category": "technology", "date": "2024-01-05"}
            ),
            Document(
                page_content="今天天气很好，适合出门散步和运动",
                metadata={"source": "weather.txt", "category": "daily", "date": "2024-01-06"}
            ),
            Document(
                page_content="股票市场今天表现良好，科技股领涨",
                metadata={"source": "market.txt", "category": "finance", "date": "2024-01-07"}
            )
        ]

        # 1.1 基础FAISS创建
        print("\n1.1 基础FAISS向量存储创建")
        faiss_vectorstore = FAISS.from_documents(documents, embeddings)
        print(f"✅ FAISS向量存储创建成功，包含 {len(documents)} 个文档")

        # 1.2 保存和加载
        print("\n1.2 FAISS索引保存和加载")
        index_path = "faiss_index"
        faiss_vectorstore.save_local(index_path)
        print(f"✅ FAISS索引已保存到 {index_path}")

        # 加载索引
        loaded_vectorstore = FAISS.load_local(
            index_path,
            embeddings,
            allow_dangerous_deserialization=True
        )
        print("✅ FAISS索引加载成功")

        # 1.3 基础相似性搜索
        print("\n1.3 基础相似性搜索")
        query = "什么是人工智能技术？"
        similar_docs = faiss_vectorstore.similarity_search(query, k=3)

        print(f"查询: '{query}'")
        print("最相似的文档:")
        for i, doc in enumerate(similar_docs):
            print(f"{i+1}. {doc.page_content}")
            print(f"   元数据: {doc.metadata}")
            print()

        # 1.4 带分数的相似性搜索
        print("\n1.4 带分数的相似性搜索")
        similar_docs_with_scores = faiss_vectorstore.similarity_search_with_score(query, k=3)

        print("带分数的搜索结果:")
        for i, (doc, score) in enumerate(similar_docs_with_scores):
            print(f"{i+1}. 分数: {score:.4f}")
            print(f"   内容: {doc.page_content[:80]}...")
            print(f"   来源: {doc.metadata.get('source', 'unknown')}")
            print()

        # 1.5 基于阈值的搜索
        print("\n1.5 基于阈值的相似性搜索")
        threshold = 0.8
        similar_docs_threshold = faiss_vectorstore.similarity_search_with_score_threshold(
            query,
            score_threshold=threshold
        )

        print(f"阈值 {threshold} 以上的文档:")
        for doc, score in similar_docs_threshold:
            print(f"分数: {score:.4f} - {doc.page_content[:60]}...")

        # 1.6 添加新文档
        print("\n1.6 添加新文档")
        new_documents = [
            Document(
                page_content="量子计算是一种利用量子力学原理的计算方式",
                metadata={"source": "quantum.txt", "category": "technology", "date": "2024-01-08"}
            ),
            Document(
                page_content="区块链技术提供了去中心化的数据存储方案",
                metadata={"source": "blockchain.txt", "category": "technology", "date": "2024-01-09"}
            )
        ]

        # 添加文档
        faiss_vectorstore.add_documents(new_documents)
        print(f"✅ 已添加 {len(new_documents)} 个新文档")

        # 验证添加结果
        new_query = "量子计算的原理"
        new_results = faiss_vectorstore.similarity_search(new_query, k=2)
        print(f"新查询 '{new_query}' 的结果:")
        for doc in new_results:
            print(f"- {doc.page_content[:50]}...")

        # 1.7 不同搜索算法
        print("\n1.7 不同FAISS搜索算法")

        # 使用不同的索引类型
        try:
            # 创建IVF索引（适合大规模数据）
            faiss_ivf = FAISS.from_documents(
                documents,
                embeddings,
                index_type="IVF",
                nlist=10  # 聚类中心数量
            )

            ivf_results = faiss_ivf.similarity_search("机器学习算法", k=2)
            print("IVF索引搜索结果:")
            for doc in ivf_results:
                print(f"- {doc.page_content[:50]}...")

        except Exception as e:
            print(f"IVF索引创建失败: {e}")

        return faiss_vectorstore

    except Exception as e:
        print(f"FAISS示例失败: {e}")
        return None
faiss_vectorstore_example()

1. FAISS向量存储示例

1.1 基础FAISS向量存储创建
✅ FAISS向量存储创建成功，包含 7 个文档

1.2 FAISS索引保存和加载
✅ FAISS索引已保存到 faiss_index
✅ FAISS索引加载成功

1.3 基础相似性搜索
查询: '什么是人工智能技术？'
最相似的文档:
1. 人工智能是计算机科学的一个分支，致力于创建智能机器
   元数据: {'source': 'ai_intro.txt', 'category': 'technology', 'date': '2024-01-01'}

2. 机器学习是人工智能的子集，使计算机能够从数据中学习
   元数据: {'source': 'ml_basics.txt', 'category': 'technology', 'date': '2024-01-02'}

3. 自然语言处理让计算机理解和生成人类语言
   元数据: {'source': 'nlp_overview.txt', 'category': 'technology', 'date': '2024-01-04'}


1.4 带分数的相似性搜索
带分数的搜索结果:
1. 分数: 0.3241
   内容: 人工智能是计算机科学的一个分支，致力于创建智能机器...
   来源: ai_intro.txt

2. 分数: 0.3984
   内容: 机器学习是人工智能的子集，使计算机能够从数据中学习...
   来源: ml_basics.txt

3. 分数: 0.6391
   内容: 自然语言处理让计算机理解和生成人类语言...
   来源: nlp_overview.txt


1.5 基于阈值的相似性搜索
FAISS示例失败: 'FAISS' object has no attribute 'similarity_search_with_score_threshold'


# Chroma向量存储详细示例

In [ ]:

def chroma_vectorstore_example():
    """Chroma向量存储详细示例"""
    print("\n" + "=" * 60)
    print("2. Chroma向量存储示例")
    print("=" * 60)

    try:
        # 初始化嵌入模型
        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )

        # 准备文档
        documents = [
            Document(
                page_content="Python是一种高级编程语言，语法简洁易读",
                metadata={"language": "python", "difficulty": "beginner", "topic": "programming"}
            ),
            Document(
                page_content="JavaScript是Web开发的核心语言之一",
                metadata={"language": "javascript", "difficulty": "intermediate", "topic": "web"}
            ),
            Document(
                page_content="Java是一种面向对象的编程语言，广泛用于企业开发",
                metadata={"language": "java", "difficulty": "intermediate", "topic": "enterprise"}
            ),
            Document(
                page_content="C++是一种高性能的系统编程语言",
                metadata={"language": "cpp", "difficulty": "advanced", "topic": "system"}
            ),
            Document(
                page_content="Go语言是Google开发的现代编程语言，适合并发编程",
                metadata={"language": "go", "difficulty": "intermediate", "topic": "concurrent"}
            )
        ]

        # 2.1 基础Chroma创建
        print("\n2.1 基础Chroma向量存储创建")
        persist_directory = "./chroma_db"

        chroma_vectorstore = Chroma.from_documents(
            documents=documents,
            embedding=embeddings,
            persist_directory=persist_directory,
            collection_name="programming_languages"
        )

        print(f"✅ Chroma向量存储创建成功，持久化目录: {persist_directory}")

        # 2.2 持久化
        print("\n2.2 Chroma数据持久化")
        chroma_vectorstore.persist()
        print("✅ Chroma数据已持久化")

        # 2.3 基础搜索
        print("\n2.3 基础相似性搜索")
        query = "适合初学者的编程语言"
        results = chroma_vectorstore.similarity_search(query, k=3)

        print(f"查询: '{query}'")
        for i, doc in enumerate(results):
            print(f"{i+1}. {doc.page_content}")
            print(f"   元数据: {doc.metadata}")
            print()

        # 2.4 元数据过滤搜索
        print("\n2.4 基于元数据的过滤搜索")

        # 搜索初学者难度的语言
        beginner_results = chroma_vectorstore.similarity_search(
            query="编程语言",
            k=5,
            filter={"difficulty": "beginner"}
        )

        print("初学者难度的编程语言:")
        for doc in beginner_results:
            print(f"- {doc.page_content}")
            print(f"  难度: {doc.metadata['difficulty']}")

        # 搜索Web相关的语言
        web_results = chroma_vectorstore.similarity_search(
            query="开发语言",
            k=5,
            filter={"topic": "web"}
        )

        print("\nWeb开发相关的语言:")
        for doc in web_results:
            print(f"- {doc.page_content}")
            print(f"  主题: {doc.metadata['topic']}")

        # 2.5 复杂过滤条件
        print("\n2.5 复杂过滤条件")

        # 使用$in操作符
        intermediate_results = chroma_vectorstore.similarity_search(
            query="编程",
            k=5,
            filter={"difficulty": {"$in": ["intermediate", "advanced"]}}
        )

        print("中级和高级难度的语言:")
        for doc in intermediate_results:
            print(f"- {doc.page_content}")
            print(f"  难度: {doc.metadata['difficulty']}")

        # 2.6 获取集合信息
        print("\n2.6 Chroma集合信息")
        collection = chroma_vectorstore._collection
        print(f"集合名称: {collection.name}")
        print(f"文档数量: {collection.count()}")

        # 2.7 删除文档
        print("\n2.7 删除文档操作")

        # 添加一个临时文档用于删除演示
        temp_doc = Document(
            page_content="临时测试文档，将被删除",
            metadata={"temp": True, "id": "temp_doc_1"}
        )

        doc_ids = chroma_vectorstore.add_documents([temp_doc])
        print(f"添加临时文档，ID: {doc_ids}")

        # 删除文档
        if doc_ids:
            chroma_vectorstore.delete(doc_ids)
            print("✅ 临时文档已删除")

        # 2.8 从现有集合加载
        print("\n2.8 从现有集合加载")

        # 创建新的Chroma实例连接到现有集合
        existing_chroma = Chroma(
            persist_directory=persist_directory,
            embedding_function=embeddings,
            collection_name="programming_languages"
        )

        # 验证加载成功
        test_results = existing_chroma.similarity_search("Python", k=1)
        print(f"从现有集合加载成功，测试搜索结果: {len(test_results)} 个文档")

        return chroma_vectorstore

    except Exception as e:
        print(f"Chroma示例失败: {e}")
        return None

# Qdrant向量存储示例

In [ ]:

def qdrant_vectorstore_example():
    """Qdrant向量存储示例"""
    print("\n" + "=" * 60)
    print("3. Qdrant向量存储示例")
    print("=" * 60)

    try:
        from qdrant_client import QdrantClient
        from qdrant_client.models import Distance, VectorParams

        # 初始化嵌入模型
        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )

        # 3.1 本地Qdrant设置
        print("\n3.1 本地Qdrant向量存储")

        # 创建Qdrant客户端（内存模式）
        client = QdrantClient(":memory:")

        # 准备文档
        documents = [
            Document(
                page_content="机器学习算法可以分为监督学习、无监督学习和强化学习",
                metadata={"category": "ml", "type": "algorithm", "level": "intermediate"}
            ),
            Document(
                page_content="监督学习使用标记数据训练模型进行预测",
                metadata={"category": "ml", "type": "supervised", "level": "beginner"}
            ),
            Document(
                page_content="无监督学习从未标记数据中发现隐藏模式",
                metadata={"category": "ml", "type": "unsupervised", "level": "intermediate"}
            ),
            Document(
                page_content="强化学习通过与环境交互来学习最优策略",
                metadata={"category": "ml", "type": "reinforcement", "level": "advanced"}
            ),
            Document(
                page_content="神经网络是深度学习的基础架构",
                metadata={"category": "dl", "type": "architecture", "level": "intermediate"}
            )
        ]

        # 创建Qdrant向量存储
        collection_name = "ml_knowledge"
        qdrant_vectorstore = Qdrant.from_documents(
            documents,
            embeddings,
            client=client,
            collection_name=collection_name,
            force_recreate=True
        )

        print(f"✅ Qdrant向量存储创建成功，集合: {collection_name}")

        # 3.2 基础搜索
        print("\n3.2 Qdrant相似性搜索")
        query = "什么是监督学习？"
        results = qdrant_vectorstore.similarity_search(query, k=3)

        print(f"查询: '{query}'")
        for i, doc in enumerate(results):
            print(f"{i+1}. {doc.page_content}")
            print(f"   类别: {doc.metadata.get('category')}")
            print(f"   类型: {doc.metadata.get('type')}")
            print()

        # 3.3 带分数搜索
        print("\n3.3 Qdrant带分数搜索")
        results_with_scores = qdrant_vectorstore.similarity_search_with_score(query, k=3)

        for i, (doc, score) in enumerate(results_with_scores):
            print(f"{i+1}. 分数: {score:.4f}")
            print(f"   内容: {doc.page_content[:60]}...")
            print()

        # 3.4 过滤搜索
        print("\n3.4 Qdrant过滤搜索")

        # 搜索特定类别
        ml_results = qdrant_vectorstore.similarity_search(
            query="学习算法",
            k=5,
            filter={"category": "ml"}
        )

        print("机器学习类别的文档:")
        for doc in ml_results:
            print(f"- {doc.page_content[:50]}...")
            print(f"  类型: {doc.metadata.get('type')}")

        # 3.5 添加新文档
        print("\n3.5 添加新文档到Qdrant")
        new_docs = [
            Document(
                page_content="卷积神经网络特别适合图像处理任务",
                metadata={"category": "dl", "type": "cnn", "level": "intermediate"}
            ),
            Document(
                page_content="循环神经网络擅长处理序列数据",
                metadata={"category": "dl", "type": "rnn", "level": "intermediate"}
            )
        ]

        qdrant_vectorstore.add_documents(new_docs)
        print(f"✅ 已添加 {len(new_docs)} 个新文档")

        # 验证添加
        cnn_results = qdrant_vectorstore.similarity_search("卷积神经网络", k=1)
        print(f"验证添加: {cnn_results[0].page_content[:40]}...")

        return qdrant_vectorstore

    except ImportError:
        print("需要安装qdrant-client: pip install qdrant-client")
        return None
    except Exception as e:
        print(f"Qdrant示例失败: {e}")
        return None

# Pinecone向量存储示例

In [ ]:

def pinecone_vectorstore_example():
    """Pinecone向量存储示例"""
    print("\n" + "=" * 60)
    print("4. Pinecone向量存储示例")
    print("=" * 60)

    try:
        import pinecone

        # 4.1 Pinecone初始化
        print("\n4.1 Pinecone初始化")

        api_key = os.getenv("PINECONE_API_KEY")
        if not api_key:
            print("请设置PINECONE_API_KEY环境变量")
            return None

        # 初始化Pinecone
        pinecone.init(
            api_key=api_key,
            environment=os.getenv("PINECONE_ENV", "us-west1-gcp")
        )

        # 初始化嵌入模型
        embeddings = OpenAIEmbeddings(
            api_key=os.getenv("OPENAI_API_KEY")
        )

        # 准备文档
        documents = [
            Document(
                page_content="云计算提供按需访问的计算资源",
                metadata={"service": "cloud", "provider": "general", "type": "infrastructure"}
            ),
            Document(
                page_content="AWS是亚马逊提供的云计算平台",
                metadata={"service": "cloud", "provider": "aws", "type": "platform"}
            ),
            Document(
                page_content="Azure是微软的云计算服务",
                metadata={"service": "cloud", "provider": "microsoft", "type": "platform"}
            ),
            Document(
                page_content="Google Cloud Platform提供各种云服务",
                metadata={"service": "cloud", "provider": "google", "type": "platform"}
            )
        ]

        # 4.2 创建Pinecone索引
        print("\n4.2 创建Pinecone索引")

        index_name = "langchain-demo"
        dimension = 1536  # OpenAI嵌入维度

        # 检查索引是否存在
        if index_name not in pinecone.list_indexes():
            pinecone.create_index(
                name=index_name,
                dimension=dimension,
                metric="cosine"
            )
            print(f"✅ 创建Pinecone索引: {index_name}")
        else:
            print(f"✅ 使用现有Pinecone索引: {index_name}")

        # 4.3 创建Pinecone向量存储
        print("\n4.3 创建Pinecone向量存储")

        pinecone_vectorstore = Pinecone.from_documents(
            documents,
            embeddings,
            index_name=index_name
        )

        print("✅ Pinecone向量存储创建成功")

        # 4.4 搜索测试
        print("\n4.4 Pinecone搜索测试")

        query = "什么是云计算平台？"
        results = pinecone_vectorstore.similarity_search(query, k=3)

        print(f"查询: '{query}'")
        for i, doc in enumerate(results):
            print(f"{i+1}. {doc.page_content}")
            print(f"   提供商: {doc.metadata.get('provider')}")
            print()

        # 4.5 命名空间使用
        print("\n4.5 Pinecone命名空间")

        # 使用命名空间分离不同类型的数据
        namespace_vectorstore = Pinecone.from_existing_index(
            index_name=index_name,
            embedding=embeddings,
            namespace="cloud_services"
        )

        # 添加文档到特定命名空间
        namespace_docs = [
            Document(
                page_content="Kubernetes是容器编排平台",
                metadata={"type": "orchestration", "category": "devops"}
            )
        ]

        namespace_vectorstore.add_documents(namespace_docs)
        print("✅ 文档已添加到命名空间")

        return pinecone_vectorstore

    except ImportError:
        print("需要安装pinecone-client: pip install pinecone-client")
        return None
    except Exception as e:
        print(f"Pinecone示例失败: {e}")
        return None

# Elasticsearch向量存储示例

In [ ]:

def elasticsearch_vectorstore_example():
    """Elasticsearch向量存储示例"""
    print("\n" + "=" * 60)
    print("5. Elasticsearch向量存储示例")
    print("=" * 60)

    try:
        from elasticsearch import Elasticsearch

        # 5.1 Elasticsearch连接
        print("\n5.1 Elasticsearch连接")

        # 连接到本地Elasticsearch
        es_client = Elasticsearch(
            [{"host": "localhost", "port": 9200}],
            # 如果有认证，添加以下配置
            # http_auth=("username", "password"),
            # use_ssl=True,
            # verify_certs=True
        )

        # 检查连接
        if not es_client.ping():
            print("无法连接到Elasticsearch，请确保服务正在运行")
            return None

        print("✅ Elasticsearch连接成功")

        # 初始化嵌入模型
        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )

        # 准备文档
        documents = [
            Document(
                page_content="搜索引擎技术是信息检索的核心",
                metadata={"domain": "search", "complexity": "medium", "year": 2024}
            ),
            Document(
                page_content="全文搜索可以在大量文档中快速找到相关内容",
                metadata={"domain": "search", "complexity": "low", "year": 2024}
            ),
            Document(
                page_content="倒排索引是搜索引擎的基础数据结构",
                metadata={"domain": "search", "complexity": "high", "year": 2024}
            ),
            Document(
                page_content="分布式搜索可以处理大规模数据集",
                metadata={"domain": "distributed", "complexity": "high", "year": 2024}
            )
        ]

        # 5.2 创建Elasticsearch向量存储
        print("\n5.2 创建Elasticsearch向量存储")

        index_name = "langchain_demo"

        es_vectorstore = ElasticsearchStore.from_documents(
            documents,
            embeddings,
            es_connection=es_client,
            index_name=index_name,
            distance_strategy="COSINE"
        )

        print(f"✅ Elasticsearch向量存储创建成功，索引: {index_name}")

        # 5.3 基础搜索
        print("\n5.3 Elasticsearch搜索")

        query = "如何实现快速搜索？"
        results = es_vectorstore.similarity_search(query, k=3)

        print(f"查询: '{query}'")
        for i, doc in enumerate(results):
            print(f"{i+1}. {doc.page_content}")
            print(f"   领域: {doc.metadata.get('domain')}")
            print(f"   复杂度: {doc.metadata.get('complexity')}")
            print()

        # 5.4 混合搜索（向量+关键词）
        print("\n5.4 Elasticsearch混合搜索")

        # 结合向量搜索和关键词搜索
        hybrid_results = es_vectorstore.similarity_search(
            query="搜索引擎",
            k=3,
            filter={"term": {"metadata.domain.keyword": "search"}}
        )

        print("混合搜索结果:")
        for doc in hybrid_results:
            print(f"- {doc.page_content}")
            print(f"  复杂度: {doc.metadata.get('complexity')}")

        # 5.5 聚合查询
        print("\n5.5 Elasticsearch聚合查询")

        # 获取不同复杂度的文档数量
        agg_query = {
            "aggs": {
                "complexity_count": {
                    "terms": {
                        "field": "metadata.complexity.keyword"
                    }
                }
            }
        }

        # 执行聚合查询
        agg_results = es_client.search(
            index=index_name,
            body=agg_query,
            size=0
        )

        print("复杂度分布:")
        for bucket in agg_results["aggregations"]["complexity_count"]["buckets"]:
            print(f"- {bucket['key']}: {bucket['doc_count']} 个文档")

        return es_vectorstore

    except ImportError:
        print("需要安装elasticsearch: pip install elasticsearch")
        return None
    except Exception as e:
        print(f"Elasticsearch示例失败: {e}")
        return None

# Redis向量存储示例

In [ ]:

def redis_vectorstore_example():
    """Redis向量存储示例"""
    print("\n" + "=" * 60)
    print("6. Redis向量存储示例")
    print("=" * 60)

    try:
        import redis

        # 6.1 Redis连接
        print("\n6.1 Redis连接")

        redis_client = redis.Redis(
            host="localhost",
            port=6379,
            db=0,
            decode_responses=True
        )

        # 检查连接
        redis_client.ping()
        print("✅ Redis连接成功")

        # 初始化嵌入模型
        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )

        # 准备文档
        documents = [
            Document(
                page_content="Redis是一个高性能的键值存储数据库",
                metadata={"type": "database", "performance": "high", "category": "nosql"}
            ),
            Document(
                page_content="Redis支持多种数据结构，如字符串、列表、集合等",
                metadata={"type": "database", "feature": "data_structures", "category": "nosql"}
            ),
            Document(
                page_content="Redis可以用作缓存、消息队列和会话存储",
                metadata={"type": "database", "use_case": "cache", "category": "nosql"}
            ),
            Document(
                page_content="Redis集群提供了高可用性和水平扩展能力",
                metadata={"type": "database", "feature": "clustering", "category": "nosql"}
            )
        ]

        # 6.2 创建Redis向量存储
        print("\n6.2 创建Redis向量存储")

        index_name = "redis_langchain_demo"

        redis_vectorstore = Redis.from_documents(
            documents,
            embeddings,
            redis_url="redis://localhost:6379",
            index_name=index_name
        )

        print(f"✅ Redis向量存储创建成功，索引: {index_name}")

        # 6.3 搜索测试
        print("\n6.3 Redis搜索测试")

        query = "什么是高性能数据库？"
        results = redis_vectorstore.similarity_search(query, k=3)

        print(f"查询: '{query}'")
        for i, doc in enumerate(results):
            print(f"{i+1}. {doc.page_content}")
            print(f"   类型: {doc.metadata.get('type')}")
            print(f"   特性: {doc.metadata.get('feature', doc.metadata.get('performance', 'N/A'))}")
            print()

        # 6.4 过滤搜索
        print("\n6.4 Redis过滤搜索")

        # 搜索特定特性的文档
        feature_results = redis_vectorstore.similarity_search(
            query="Redis功能",
            k=5,
            filter={"feature": "data_structures"}
        )

        print("数据结构相关的文档:")
        for doc in feature_results:
            print(f"- {doc.page_content}")

        return redis_vectorstore

    except ImportError:
        print("需要安装redis: pip install redis")
        return None
    except Exception as e:
        print(f"Redis示例失败: {e}")
        return None

# 向量存储性能对比

In [ ]:

def vector_store_comparison():
    """向量存储性能对比"""
    print("\n" + "=" * 60)
    print("7. 向量存储性能对比")
    print("=" * 60)

    # 准备测试数据
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )

    # 生成测试文档
    test_documents = []
    for i in range(100):
        doc = Document(
            page_content=f"这是第{i}个测试文档，包含一些示例内容用于性能测试。文档编号：{i}",
            metadata={"doc_id": i, "category": f"cat_{i % 5}", "priority": i % 3}
        )
        test_documents.append(doc)

    test_query = "测试文档内容"

    # 测试不同向量存储的性能
    stores_to_test = []

    # FAISS测试
    try:
        print("\n7.1 FAISS性能测试")
        start_time = time.time()
        faiss_store = FAISS.from_documents(test_documents, embeddings)
        creation_time = time.time() - start_time

        start_time = time.time()
        faiss_results = faiss_store.similarity_search(test_query, k=5)
        search_time = time.time() - start_time

        stores_to_test.append({
            "name": "FAISS",
            "creation_time": creation_time,
            "search_time": search_time,
            "results_count": len(faiss_results)
        })

        print(f"FAISS - 创建时间: {creation_time:.4f}s, 搜索时间: {search_time:.4f}s")

    except Exception as e:
        print(f"FAISS测试失败: {e}")

    # Chroma测试
    try:
        print("\n7.2 Chroma性能测试")
        start_time = time.time()
        chroma_store = Chroma.from_documents(
            test_documents,
            embeddings,
            persist_directory="./test_chroma",
            collection_name="performance_test"
        )
        creation_time = time.time() - start_time

        start_time = time.time()
        chroma_results = chroma_store.similarity_search(test_query, k=5)
        search_time = time.time() - start_time

        stores_to_test.append({
            "name": "Chroma",
            "creation_time": creation_time,
            "search_time": search_time,
            "results_count": len(chroma_results)
        })

        print(f"Chroma - 创建时间: {creation_time:.4f}s, 搜索时间: {search_time:.4f}s")

    except Exception as e:
        print(f"Chroma测试失败: {e}")

    # 性能对比总结
    print("\n7.3 性能对比总结")
    print(f"{'存储类型':<15} {'创建时间(s)':<12} {'搜索时间(s)':<12} {'结果数量':<10}")
    print("-" * 55)

    for store in stores_to_test:
        print(f"{store['name']:<15} {store['creation_time']:<12.4f} "
              f"{store['search_time']:<12.4f} {store['results_count']:<10}")

    # 推荐建议
    print("\n7.4 选择建议")
    print("📋 向量存储选择指南:")
    print("1. 小规模本地应用: FAISS")
    print("2. 需要持久化: Chroma")
    print("3. 生产环境大规模: Pinecone/Qdrant")
    print("4. 已有Elasticsearch: ElasticsearchStore")
    print("5. 需要高性能缓存: Redis")
    print("6. 企业级应用: Weaviate/Milvus")

# 高级向量操作示例

In [ ]:

def advanced_vector_operations():
    """高级向量操作示例"""
    print("\n" + "=" * 60)
    print("8. 高级向量操作示例")
    print("=" * 60)

    try:
        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )

        # 8.1 向量存储合并
        print("\n8.1 向量存储合并")

        # 创建两个不同的向量存储
        docs1 = [
            Document(page_content="第一个存储的文档1", metadata={"store": "store1"}),
            Document(page_content="第一个存储的文档2", metadata={"store": "store1"})
        ]

        docs2 = [
            Document(page_content="第二个存储的文档1", metadata={"store": "store2"}),
            Document(page_content="第二个存储的文档2", metadata={"store": "store2"})
        ]

        store1 = FAISS.from_documents(docs1, embeddings)
        store2 = FAISS.from_documents(docs2, embeddings)

        # 合并向量存储
        store1.merge_from(store2)
        print("✅ 向量存储合并完成")

        # 验证合并结果
        all_results = store1.similarity_search("文档", k=4)
        print(f"合并后总文档数: {len(all_results)}")
        for doc in all_results:
            print(f"- {doc.page_content} (来源: {doc.metadata['store']})")

        # 8.2 批量操作
        print("\n8.2 批量向量操作")

        # 批量添加文档
        batch_docs = [
            Document(page_content=f"批量文档{i}", metadata={"batch": True, "id": i})
            for i in range(10)
        ]

        start_time = time.time()
        store1.add_documents(batch_docs)
        batch_time = time.time() - start_time

        print(f"批量添加10个文档耗时: {batch_time:.4f}s")

        # 8.3 向量存储统计
        print("\n8.3 向量存储统计信息")

        # 获取所有文档进行统计
        all_docs = store1.similarity_search("", k=100)  # 获取更多文档

        # 统计不同来源的文档数量
        store_counts = {}
        batch_count = 0

        for doc in all_docs:
            if doc.metadata.get("batch"):
                batch_count += 1
            else:
                store = doc.metadata.get("store", "unknown")
                store_counts[store] = store_counts.get(store, 0) + 1

        print("文档统计:")
        for store, count in store_counts.items():
            print(f"- {store}: {count} 个文档")
        print(f"- 批量文档: {batch_count} 个文档")

        # 8.4 相似度阈值过滤
        print("\n8.4 相似度阈值过滤")

        query = "批量文档"
        threshold = 0.5

        # 获取所有结果并手动过滤
        all_results_with_scores = store1.similarity_search_with_score(query, k=20)

        filtered_results = [
            (doc, score) for doc, score in all_results_with_scores
            if score <= threshold  # FAISS使用距离，越小越相似
        ]

        print(f"阈值 {threshold} 以下的结果:")
        for doc, score in filtered_results[:5]:
            print(f"- 分数: {score:.4f} - {doc.page_content}")

        return store1

    except Exception as e:
        print(f"高级向量操作失败: {e}")
        return None

In [ ]:

def main():
    """运行所有向量存储示例"""
    print("🚀 LangChain 0.3 Vector Stores 完整示例")
    print("=" * 80)

    # 运行所有示例
    faiss_store = faiss_vectorstore_example()
    chroma_store = chroma_vectorstore_example()
    qdrant_store = qdrant_vectorstore_example()
    pinecone_store = pinecone_vectorstore_example()
    es_store = elasticsearch_vectorstore_example()
    redis_store = redis_vectorstore_example()

    # 性能对比
    vector_store_comparison()

    # 高级操作
    advanced_vector_operations()

    print("\n🎉 所有向量存储示例运行完成！")

    # 清理临时文件
    import shutil
    temp_dirs = ["faiss_index", "chroma_db", "test_chroma"]
    for temp_dir in temp_dirs:
        if os.path.exists(temp_dir):
            try:
                shutil.rmtree(temp_dir)
                print(f"🧹 已清理临时目录: {temp_dir}")
            except Exception as e:
                print(f"清理 {temp_dir} 失败: {e}")

if __name__ == "__main__":
    main()